# Julia nivel intermedio

## Parte 1

Hemos visto varias cosas:

- asignación de variables, incluyendo diversos tipos numéricos(`Int64`, `Float64`, `BigFloat`, `Complex{Float64}`, `Rational{Int64}`, etc), cadenas, vectores, matrices

- ciclos (`for`...`end`; `while`...`end`) y rangos (`a:b`)

- diversas formas de condicionales

- funciones simples

Aquí seguiremos con esto, introduciendo ahora el concepto de funciones paramétricas, la forma de definir tipos arbitrarios y cómo escribir código que escribe o modifica código.

## Ejemplo: el método de Newton

Como bien sabemos, el método de Newton es un método iterativo para encontrar los ceros, o raíces, de una ecuación $f(x)=0$. A partir de una aproximación $x_0$, y denotando la derivada de $f(x)$ como $f'(x)$, tenemos:

\begin{equation}
x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)} .
\end{equation}



Para ejemplificar las cosas usaremos la función $f(x) = x^2-2$:

In [ ]:
f(x) = x^2 - 2

... y su derivada:

In [ ]:
f´(x) = 2*x

Entonces, el método de Newton vendría dado por el siguiente código:

In [ ]:
# Condición inicial para iterar el método
x_0 = 3.0

for i in 1:10  # Consideramos 10 iteraciones del método
    x_0 = x_0 - f(x_0) / f´(x_0)
    println(i, "\t", x_0)
end
x_0

El error relativo es:

In [ ]:
(x_0-sqrt(2.0))/sqrt(2.0)

Recordando la clase de cálculo I, el método de Newton converge si $x_0$ es *suficientemente* cercano a la raíz. Veamos qué pasa si *variamos* la condición inicial $x_0$.

In [ ]:
# Consideramos condiciones iniciales desde -3 a 3, con pasos de 0.125
condicion_inicial = -3:0.125:3

In [ ]:
typeof(condicion_inicial)

In [ ]:
collect(condicion_inicial)[1:5]   # primeros 5 elementos de `condicion_inicial`

Defino el arreglo de salida `raices` que, entrada por entrada, corresponderá a las condiciones iniciales definidas en `condicion_inicial`. Esto es conveniente ya que así separo la memoria que necesitaré.

In [ ]:
# Inicializa los valores de `raices`, cuya longitud es la misma que `condicion_inicial`; el default es `Float64`
raices = zeros(length(condicion_inicial))

In [ ]:
# `similar` copia la estructura de `condicion_inicial` (tipo y longitud) en `raices`
raices = similar(condicion_inicial)

El número de datos de `condicion_inicial` es claramente 49; otra manera de ver esto es:

In [ ]:
length(condicion_inicial)

Ahora, implementamos todo junto; abajo, `enumerate(initial_conditions)` regresa una *tupla* con un entero, que es el índice de la condición inicial (`j`), y su valor (`x_0`)

In [ ]:
for (j, x_0) in enumerate(condicion_inicial)
    x = x_0

    # 100 iterates of Newton's method
    for i in 1:100
        x = x - f(x) / f´(x)
    end
    
    raices[j] = x
end

Veamos los resultados:

In [ ]:
# ";" suprime la salida (el vector es demasiado largo); en este caso, suprime muchos `nothing`
map(println, raices);

**Pregunta:** ¿Por qué hay un `NaN` entre las raíces obtenidas?

In [ ]:
raices[25]

Ahora vamos a visualizar esto. Para eso usaremos el paquete "PyPlot", que es una interface para usar "matplotlb"; más adelante usaremos otras paqueterías.

Para instalarlo es necesario usar el siguiente comando:
```julia
    Pkg.add("PyPlot")
```

Otros paquetes *registrados* se pueden encontrar [aquí](http://pkg.julialang.org/); la lista empieza a ser larga!

Para usar un paquete ya instalado, en este caso "PyPlot", usamos el comando:

```julia
    using PyPlot
```

(En este caso, tardará un poco la primera vez que se utiliza.)

In [ ]:
using PyPlot

In [ ]:
# este comando define el tamaño de la figura
figure(figsize=(6,4))

plot(raices)           # este comando pinta las componentes "y" (`raices`); 
                      # en "x" se utiliza el índice del vector

In [ ]:
figure(figsize=(6,4)) # este comando define el tamaño de la figura

plot(condicion_inicial, raices)    # este comando pinta las componentes "y" (`raices`); 
                                # en "x" se utiliza `condicion_inicial`
                                # Los vectores *tienen* que tener el mismo número de elementos

title(L"Convergencia del método de Newton para $f(x)=x^2-2$", fontsize=10)

xlabel(L"$x_0$ (Condición inicial)")
ylabel(L"$x_{100}$ (Iterado de Newton final)")

## Rendimiento

La manera en que procedimos arriba **no** es la óptima para trabajar en Julia. Julia es rápido cuando uno **no** trabaja en el *global scope*, que es lo que hicimos; es **mucho** mejor poner el código que será utilizado de manera repetida dentro de una función.

Entonces, lo anterior, lo empacamos dentro de una función.

In [ ]:
"""
    calcula_raices()

Esta función calcula las raices de una función `f(x)`, que debe haberse definido,
considerando distintos valores de las condiciones iniciales en el intervalo 
\$ x\\in [-20,20] \$ , y devuelve los resultados obtenidos en un vector.
"""
function calcula_raices()
    condiciones_iniciales = -20:0.125:20
    raices = similar(condiciones_iniciales)

    for (j, x_0) in enumerate(condiciones_iniciales)
        x = x_0

        # 100 iterations of Newton's method
        for i in 1:100
            x = x - f(x) / f´(x)
        end

        raices[j] = x
    end
    
    raices
end

Noten la salida de ejecutar la definición de la función `calcula_raices`:

```
    calcula_raices (generic function with 1 method)
```

A esto volveremos más adelante...

In [ ]:
?calcula_raices

**Ejercicio 1:** ¿Cuál es el tipo de `condiciones_iniciales` y cual es el de `raices`?

In [ ]:
raices = calcula_raices();

¿Cuánto tiempo tardó esto?

El *macro* `@time` (que usamos la clase pasada) permite precisamente medir el tiempo de ejecución y el uso de memoria.

Julia *compila* cada función la primera vez que es utilizada. Es por eso que conviene compilar `@time`, y luego la ejecución de `@time` con la función que nos interesa medir. Así, para medir de manera consistente el tiempo de ejecución (modulo pequeñas fluctuaciones), es importante no incluir en esa medición la compilación de la función.

In [ ]:
@time 1;

In [ ]:
@time 1;

Las dos ejecuciones anteriores muestran que, al compilar (primer ejecución), julia utiliza espacio de memoria; una vez compiladas las cosas, el uso de memoria disminuye de manera importante. Incidentalmente, el uso exagerado de memoria muestra que las cosas pueden ser hechas de mejor manera...

In [ ]:
@time calcula_raices();

In [ ]:
@time calcula_raices();  # como `calcula_raices` ya está compilada, no hay mucho cambio en el
                       # tiempo de ejecución o memoria usada

¿Y si queremos calcular los ceros de otra función? Dada la actual implementación de `calcula_raices` tendríamos que:

- redefinir `f(x)` y `f'(x)`, y

- recompilar `calcula_raices()` (sino, utiliza la función compilada con `f(x)=x^2-2`)

## Funciones genéricas

Lo que hemos hecho hasta ahora está bien, pero *no* es muy práctico, en el sentido de que no es muy cómodo la implementación si queremos considerar distintas funciones `f(x)`.

Para esto, *definimos* el siguiente *método* para la función `calcula_raices`; noten que es *el mismo nombre* de la función que definimos antes.

In [ ]:
function calcula_raices(f, f´)
    condiciones_iniciales = -20:0.125:20
    raices = similar(condiciones_iniciales)

    for (j, x_0) in enumerate(condiciones_iniciales)
        x = x_0

        # 100 iterations of Newton's method
        for i in 1:100
            x = x - f(x) / f´(x)
        end

        raices[j] = x
    end
    
    raices
end

Es importante notar que, ahora, tenemos 2 métodos definidos para la *misma* función `calcula_raices`. Esto es, **no** hemos sobreescrito la función `calcula_raices`, sino que hemos creado un nuevo método, y cada método se aplica según el número de argumentos de la función. En este caso, la distinción es muy sencilla; uno puede hacer que los distintos métodos se distingan según el *tipo* de los parámetros.

La instrucción `methods` ayuda a saber cuántos métodos hay asociados a una función, incluyendo información sobre el tipo de los argumentos; esto último es la "signatura de tipo" (*type signature*).

In [ ]:
methods(calcula_raices)

**Ejercicio 2:** ¿De qué tipo son las variables f y f´?

Ejecutamos ahora la nueva implementación de las funciones:

In [ ]:
@time calcula_raices(f, f´);

In [ ]:
@time calcula_raices(f, f´);

Vale la pena notar que, la vieja implementación corre más o menos igual de rápido que esta implementación; esto tiene que ver con el hecho de que las funciones `f` y `f´` tienen un tipo específico.

Como vimos anteriormente, la manera de notar si una función va a dar problemas en cuanto a la estabilidad de tipo es usando el macro `@code_warntype`. 

In [ ]:
@code_warntype(calcula_raices())

In [ ]:
@code_warntype(calcula_raices(f, f´))

A partir de Julia v0.5, uno puede hacer esto mismo *sin* tener una alta penalización en el tiempo de ejecución, usando funciones anónimas es la siguiente:

In [ ]:
@time calcula_raices(x->x^2-2, x->2x);

In [ ]:
@time calcula_raices(x->x^2-2, x->2x);

¿Cómo le podemos hacer si queremos guardar el tiempo de ejecución?

In [ ]:
@elapsed calcula_raices(x->x^2-2, x->2x)

Hagamos otro ejemplo un poco más interesante:

In [ ]:
@time raices = calcula_raices( x -> (x-1)*(x-2)*(x-3), 
                             x->(x-2)*(x-3)+(x-1)*(x-3)+(x-1)*(x-2) );

In [ ]:
@time raices = calcula_raices( x -> (x-1)*(x-2)*(x-3), 
                             x->(x-2)*(x-3)+(x-1)*(x-3)+(x-1)*(x-2) );

In [ ]:
figure(figsize=(6,4))
plot(-20:0.125:20, raices)
ylim(0,4)

In [ ]:
figure(figsize=(6,4))
plot(-20:0.125:20, roots, "g.-")
ylim(0,4)
xlim(1,3)

**Ejercicio 3:** Modificar y documentar (!) la función `calcula_raices` de tal manera que la condición inicial sea un rango arbitrario que el usuario da a la entrada.

Usando esta función, ¿cómo se ve las raíces a las que converge la iteración del método de Newton para $f(x) = (x-1)(x-2)(x-3)$? En este caso vale la pena concentrarse en valores $x_0\in [1,3]$, usar muchos puntos y quizás hacer observar localmente la estructura.

**Ejercicio 4:** Usando la función que acaban de hacer para rangos arbitrarios, grafiquen *la dependencia* de las raíces respecto a las condiciones iniciales considerando los intervalos de condiciones iniciales $[1,3]$, $[1.3,1.7]$, $[1.5,1.6]$, $[1.552,1.5552]$

## Global scope

Las variables (asignaciones) que uno define en el Jupyter notebook o en el REPL, por ejemplo, ejecutando
```julia
    x = 3
```
están definidas en lo que se llama el *global scope*. Las que uno define internamente en ciclos (por ejemplo, `for... x = 4 ... end`) o en funciones, están definidas en el *local scope*, del ciclo o de la función.

Una cuestión **importante** es que Julia *penaliza* las variables definidas en el global scope; la razón es que dichas variables pueden eventualmente cambiar de tipo, por lo que deben ser guardadas como apuntadores. Ya hemos visto que si las variables cambian de tipo en el *local scope*, está castiga mucho el tiempo de ejecución.


Si uno quiere usar, de manera eficiente, variables en el *global scope*, uno debe definirlas como *constantes*:
```julia
    const x = 3
```

La variable `x` será definida como de tipo `Int` en este ejemplo. Uno puede cambiarle el valor, pero no el tipo.

In [ ]:
const var_globalscope = 3

In [ ]:
var_globalscope

In [ ]:
isconst(var_globalscope)

In [ ]:
isconst(:var_globalscope)

El hecho de que declaramos a `var_globalscope` como `const` no significa que no puede cambiar de valor; simplemente, **no** puede cambiar de tipo.

In [ ]:
var_globalscope = 0

In [ ]:
var_globalscope

In [ ]:
var_globalscope = 1.125

In [ ]:
const var_globalscope = 1.125

## El método de Newton sobre los complejos

Ahora, implementaremos el método de Newton, para alguna función modelo ($f(z) = z^3-1$), pero usaremos condiciones iniciales en los complejos.

Las condiciones iniciales, igual que antes, las definiremos a partir de un `FloatRange{Float64}`, que usaremos tanto para la parte real como para la parte imaginaria de $z_0$.

A priori podríamos proceder como antes. Sin embargo, hay *sutilezas*, ya que la salida `raices` no será un vector, sino una matriz.

In [ ]:
const cc = complex(1.0,0.0)

f(z) = z^3 - cc
f´(z) = 3*z^2

**NOTA**: La siguiente función necesita las funciones $f$ y $f'$, lo que permite usarlas en contextos más generales. Esto, como vimos antes, tiene una penalización en la ejecución.

In [ ]:
function compute_complex_roots(f, f´, range=-5.0:0.125:5.0)

    L = length(range)
    
    ## Se define la matriz con todos los elementos inicialmente en cero
    roots = zeros(Complex128, L, L)

#     for (j, y) in enumerate(range)
#         for (i, x) in enumerate(range)
    for (j, y) in enumerate(range), (i, x) in enumerate(range)
            
            z = x + y*im
            
            for k in 1:1000
                z = z - f(z) / f´(z)
            end
            
            roots[i,j] = z
            
    end
    
    roots
end

**Ejercicio 4:** ¿Por qué `compute_complex_roots` tiene 2 métodos definidos?

In [ ]:
@time compute_complex_roots(f, f´, -5.0:1.0:5.0);

In [ ]:
@time croots = compute_complex_roots(f, f´, -5.0:1/32:5.0);

Visualicemos los resultados: para esto usaremos `imshow`, que sirve para visualizar una matriz, y el código de colores lo definiremos a partir de la parte imaginaria de `roots`.

In [ ]:
imshow(imag(croots))

**NOTA** La librería `PyPlot` está basada en `matplotlib`; en la red hay muchos recursos para usar `matplotlib` y generar diversos tipos de gráficas. Recuerden, ¡google es su amigo!